In [1]:
import ipyvuetify as v
import ipywidgets as widgets

In [9]:
#for testing 
year = '2019'
asset_name = "users/bornToBeAlive/Glad_Singapore_boundaries"

In [3]:
widget_gee_process_alert = v.Alert(children=[''], type='info', text=True, class_="mt-5")
#widget_alert

In [4]:
import os
import ee
import time

#initialize earth engine
ee.Initialize()

def download_to_disk(filename, alerts, aoi_name):
    """download the tile to the GEE disk"""
    task_config = {
        'image':alerts,
        'description':filename,
        'scale': 30,
        'region':ee.FeatureCollection(aoi_name).geometry(),
        'maxPixels': 1e10
    }
    
    task = ee.batch.Export.image.toDrive(**task_config)
    task.start()
    
    return task

def get_alerts(aoi_name, year):
    """ get the alerts from the GLAD project"""
    
    aoi = ee.FeatureCollection(aoi_name)
    all_alerts  = ee.ImageCollection('projects/glad/alert/UpdResult')
    alerts = all_alerts.select('conf' + year[-2:]).mosaic().clip(aoi);
    
    return alerts

def construct_filename(asset_name, year):
    """return the filename associated with the current task"""
    aoi_name= os.path.split(asset_name)[1]
    filename = 'alerts_' + aoi_name + '_' + str(year) + "test0-0" #remove test when it will be in production 
    
    return filename
    
def run_GEE_process(asset_name, year):
    
    global widget_gee_process_alert
    
    
    #search for the task in task_list
    filename = construct_filename(asset_name, year)
    
    tasks_list = ee.batch.Task.list()
    current_task = ''
    for task in tasks_list:
        if task.config['description'] == filename:
            current_task = task
            break 
    
    #launch the task in GEE 
    if current_task == '':
        
        alerts = get_alerts(asset_name, year)
        current_task = download_to_disk(filename, alerts, asset_name)
        message_init = "the task {0} has been launched on your GEE account".format(filename)
        state = current_task.state
        while state != 'COMPLETED':
            widget_gee_process_alert.children = [message_init + " Status: " + state]
            widget_gee_process_alert.type = "info"
            #print(message_init + " Status: " + state)
            time.sleep(20)
            #search for the task in task_list
            filename = construct_filename(asset_name, year)
            tasks_list = ee.batch.Task.list()
            current_task = ''
            for task in tasks_list:
                if task.config['description'] == filename:
                    current_task = task
                    break
            state = current_task.state
        
        #print(message_init)
        return 'success', message_init 
            
    
    #give insight if it's still running 
    if current_task.state == "RUNNING":
        message_init = "the task {0} is currently running on the GEE servers ....".format(filename)
        state = current_task.state
        while state != 'COMPLETED':
            widget_gee_process_alert.children = [message_init + " Status: " + state]
            widget_gee_process_alert.type = "info"
            #print(message_init + " Status: " + state)
            time.sleep(20)
            #search for the task in task_list
            filename = construct_filename(asset_name, year)
            tasks_list = ee.batch.Task.list()
            current_task = ''
            for task in tasks_list:
                if task.config['description'] == filename:
                    current_task = task
                    break
            state = current_task.state
        #print("Task {0} completed".format(filename))
        return 'success', "Task {0} completed".format(filename) 
    
    #say when it's completed
    if current_task.state == "COMPLETED":
        message_init = 'the task {0} has already been completed and is available on your Gdrive.'.format(filename)
        #print(message_init)
        return "sucess", message_init
    

In [5]:
#validate the selected data
gee_process_btn = v.Btn(
    color='primary', 
    children=[
        v.Icon(left=True, children=['mdi-cogs']),
        'Run GEE process'
    ]
)
widget_validate_data = v.Container(children=[gee_process_btn])

def toggleLoading(btn):
    btn.loading = not btn.loading
    btn.disabled = btn.loading

def on_click(widget, event, data):
    
    global widget_gee_process_alert
    global asset_name
    global year
    global gee_process_btn
    
    if asset_name == None or year == None:
        widget_gee_process_alert.type="error"
        widget_gee_process_alert.children = ["the input are not correctly set up"]
        return 
    
    #do something 
    toggleLoading(gee_process_btn)
    alert_type, message = run_GEE_process(asset_name, year)
    toggleLoading(gee_process_btn)
    
    widget_gee_process_alert.type=alert_type
    widget_gee_process_alert.children = [message]

widget_validate_data.on_event('click', on_click)
#widget_validate_data

In [6]:
#create the display 
inputs = v.Layout(
    _metadata={'mount-id': 'data-input'},
    row=True,
    class_="pa-5",
    align_center=True, 
    children=[
        v.Flex(xs12=True, children=[widget_validate_data]),
        v.Flex(xs12=True, children=[widget_gee_process_alert])
    ]
)
#inputs

GEE_content_main =  v.Layout(
    _metadata={'mount_id': 'content-main'},
    row=True,
    xs12=True,
    align_center=True, 
    class_="ma-5",
    children=[
        v.Card( 
            class_="pa-5",
            raised=True,
            xs12=True,
            children=[
                v.Html(xs12=True, tag='h2', children=['Run the GEE process']),
                v.Flex(xs12=True, lg6=True, children=[inputs]),   
            ]
        )
    ]
)
GEE_content_main

Layout(align_center=True, children=[Card(children=[Html(children=['Run the GEE process'], tag='h2'), Flex(chil…